In [1]:
import tensorflow as tf

2025-03-16 00:04:50.318115: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-16 00:04:50.533772: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-16 00:04:50.752665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742083490.952015    6679 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742083491.018275    6679 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 00:04:51.534912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# Image processing & training model

with tf.device('/device:GPU:0'):
  # Using CIFAR10, a varied CV dataset to determine model metrics 
  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

  batch_size = 32

  def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label

  # Create training dataset
  train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
  train_ds = train_ds.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

  # Create testing dataset
  test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
  test_ds = test_ds.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

  # Build model
  base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(224, 224, 3),
    weights='imagenet',
    pooling='avg'
  )
  base_model.trainable = False

  model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  model.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds
  )



KeyboardInterrupt: 

In [ ]:
# Testing model after training
print(f"Number of training images: {len(train_images)}")
print(f"Number of testing images: {len(test_images)}")
model.evaluate(test_ds)